<a href="https://colab.research.google.com/github/xCosmicx/ATA/blob/main/week12/bert-multiclass-solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning BERT for Multi-Class Classification (Solution)

### Install Hugging Face Transformers library

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 8.2 MB/s 
     |████████████████████████████████| 596 kB 49.1 MB/s 
     |████████████████████████████████| 101 kB 12.3 MB/s 
     |████████████████████████████████| 6.6 MB 43.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

## Data Preparation

In [3]:
data_url = 'https://nyp-aicourse.s3.ap-southeast-1.amazonaws.com/it3103/news.csv'
df = pd.read_csv(data_url, delimiter='\t')

In [4]:
df.head()

,TITLE,CATEGORY
0,"Fed official says weak data caused by weather,...",b
1,Fed's Charles Plosser sees high bar for change...,b
2,US open: Stocks fall after Fed official hints ...,b
3,"Fed risks falling 'behind the curve', Charles ...",b
4,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,b


In [5]:
len(df)

422419

In [6]:
SUBSET_SIZE = 2500

subset_df = df.sample(n=SUBSET_SIZE, random_state=128)

In [7]:
subset_df['CATEGORY'].value_counts()

e    873
t    694
b    682
m    251
Name: CATEGORY, dtype: int64

In [8]:
def map_label(x):
    if x == 'e':
        return 0
    elif x == 't':
        return 1
    elif x == 'b':
        return 2
    elif x == 'm':
        return 3

labels_map = ['entertainment','tech','business','medical/health']

We now convert the text label into numeric values of 0 (negative) and 1 (positive) 

In [9]:
subset_df['CATEGORY'] =  subset_df['CATEGORY'].apply(map_label)

In [10]:
texts = subset_df['TITLE']
labels = subset_df['CATEGORY']

In [11]:
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=.2)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [12]:
len(train_texts)

1600

## Tokenization


In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [14]:
train_texts = train_texts.to_list()
train_labels = train_labels.to_list()
val_texts = val_texts.to_list()
val_labels = val_labels.to_list()
test_texts = test_texts.to_list()
test_labels = test_labels.to_list()

In [15]:
train_encodings = tokenizer(train_texts, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, padding=True, truncation=True)
test_encodings = tokenizer(test_texts, padding=True, truncation=True)

In [16]:
batch_size = 16

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
)).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
)).batch(batch_size)

## Fine-tuning the model

In [17]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased", num_labels=4)

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_transform', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [18]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

num_epochs = 2

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Since our dataset is already batched, we can simply take the len.
num_train_steps = len(train_dataset) * num_epochs

lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

In [19]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

opt = Adam(learning_rate=lr_scheduler)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

model.fit(train_dataset, validation_data=val_dataset, epochs=num_epochs)

Epoch 1/2
100/100 [==============================] - 21s 83ms/step - loss: 0.6709 - accuracy: 0.7619 - val_loss: 0.3992 - val_accuracy: 0.8675
Epoch 2/2
100/100 [==============================] - 7s 65ms/step - loss: 0.2618 - accuracy: 0.9269 - val_loss: 0.3804 - val_accuracy: 0.8700


In [20]:
model.evaluate(test_dataset)

32/32 [==============================] - 3s 27ms/step - loss: 0.3770 - accuracy: 0.8700


[0.3770335614681244, 0.8700000047683716]

In [21]:
model.save_pretrained('multiclass_model')

## Try out the model

In [22]:
my_model = TFAutoModelForSequenceClassification.from_pretrained(
        "multiclass_model")

Some layers from the model checkpoint at multiclass_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at multiclass_model and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
text = input('Write your news article here:')

Write your news article here:FANG stocks have plummet due to Russia Ukraine war


In [24]:
inputs = tokenizer(text, return_tensors="tf")
output = my_model(inputs)
pred_prob = tf.nn.softmax(output.logits, axis=-1)
pred = np.argmax(pred_prob)
print(labels_map[pred])

business
